In [1]:
import sys
sys.path.append('..')
import hydra
import torch
from experiments.video_generation import VideoGenerationExperiment
from einops import rearrange
import numpy as np
from scipy.stats import spearmanr
import cv2

/home/s224075134/diffusion-forcing-transformer/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/s224075134/diffusion-forcing-transformer/venv/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [2]:
import hydra
from hydra import compose, initialize
from omegaconf import OmegaConf
from omegaconf import DictConfig
from omegaconf.omegaconf import open_dict
# Initialize Hydra
# The version_base is optional but recommended to avoid deprecation warnings
with initialize(version_base=None, config_path="../configurations"):
    # Compose the config - replace "config_name" with your actual config name
    # You can also override values here if needed
    cfg = compose(
        config_name="config", 
        overrides=[
            'algorithm=contrastive_dfot_video', 
            "experiment=video_generation",
            'dataset=ucf_101',
            'experiment.tasks=[validation]',
            'experiment.validation.data.shuffle=False',
            'dataset.context_length=4',
            'dataset.n_frames=12',
            'experiment.validation.batch_size=4',
            'algorithm.tasks.prediction.history_guidance.name=vanilla',
            '++algorithm.backbone.name=u_net3d', 
            '++algorithm.backbone.network_size=48', 
            '++algorithm.backbone.num_res_blocks=2', 
            '++algorithm.backbone.resnet_block_groups=8', 
            '++algorithm.backbone.dim_mults=[1, 2, 4, 8]', 
            '++algorithm.backbone.attn_resolutions=[8, 16, 32, 64]', 
            '++algorithm.backbone.attn_dim_head=32', 
            '++algorithm.backbone.attn_heads=4', 
            '++algorithm.backbone.use_linear_attn=True', 
            '++algorithm.backbone.use_init_temporal_attn=True', 
            '++algorithm.backbone.init_kernel_size=7', 
            '++algorithm.backbone.dropout=0.0 ' 
            ],
        return_hydra_config=True
    )
    
    # Now you have the config in the 'cfg' variable
    # You can print it, access values, etc. without running your actual application
    # print(OmegaConf.to_yaml(cfg))
    
    # Access config values
    # print(f"Some value from config: {cfg.some_key}")
cfg_choice = cfg['hydra'].runtime.choices
with open_dict(cfg):
    if cfg_choice["experiment"] is not None:
        cfg.experiment._name = cfg_choice["experiment"]
    if cfg_choice["dataset"] is not None:
        cfg.dataset._name = cfg_choice["dataset"]
    if cfg_choice["algorithm"] is not None:
        cfg.algorithm._name = cfg_choice["algorithm"]
    

exp = VideoGenerationExperiment(cfg, None, None)
algo = exp._build_algo()
algo = algo.to('cuda:0')

/home/s224075134/diffusion-forcing-transformer/venv/lib/python3.10/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


{'hydra': {'run': {'dir': '${output_dir}/${now:%Y-%m-%d}/${now:%H-%M-%S}'}, 'sweep': {'dir': 'multirun/${now:%Y-%m-%d}/${now:%H-%M-%S}', 'subdir': '${hydra.job.num}'}, 'launcher': {'_target_': 'hydra._internal.core_plugins.basic_launcher.BasicLauncher'}, 'sweeper': {'_target_': 'hydra._internal.core_plugins.basic_sweeper.BasicSweeper', 'max_batch_size': None, 'params': None}, 'help': {'app_name': '${hydra.job.name}', 'header': '${hydra.help.app_name} is powered by Hydra.\n', 'footer': 'Powered by Hydra (https://hydra.cc)\nUse --hydra-help to view Hydra specific help\n', 'template': '${hydra.help.header}\n== Configuration groups ==\nCompose your configuration from those groups (group=option)\n\n$APP_CONFIG_GROUPS\n\n== Config ==\nOverride anything in the config (foo.bar=value)\n\n$CONFIG\n\n${hydra.help.footer}\n'}, 'hydra_help': {'template': "Hydra (${hydra.runtime.version})\nSee https://hydra.cc for more info.\n\n== Flags ==\n$FLAGS_HELP\n\n== Configuration groups ==\nCompose your con

In [3]:
x = torch.randn(4, 12, 3, 64, 64).to('cuda:0')
k = torch.randint(0, 1000, (4, 12)).to('cuda:0')
output = algo.diffusion_model.model(x, k)

/home/s224075134/diffusion-forcing-transformer/test/../algorithms/contrastive_dfot/backbones/u_net/u_net_blocks.py:166: UserWarning: Memory efficient kernel not used because: (Triggered internally at /pytorch/aten/src/ATen/native/transformers/cuda/sdp_utils.cpp:776.)
  hidden_states = F.scaled_dot_product_attention(
/home/s224075134/diffusion-forcing-transformer/test/../algorithms/contrastive_dfot/backbones/u_net/u_net_blocks.py:166: UserWarning: Memory Efficient attention has been runtime disabled. (Triggered internally at /pytorch/aten/src/ATen/native/transformers/sdp_utils_cpp.h:551.)
  hidden_states = F.scaled_dot_product_attention(
/home/s224075134/diffusion-forcing-transformer/test/../algorithms/contrastive_dfot/backbones/u_net/u_net_blocks.py:166: UserWarning: Flash attention kernel not used because: (Triggered internally at /pytorch/aten/src/ATen/native/transformers/cuda/sdp_utils.cpp:778.)
  hidden_states = F.scaled_dot_product_attention(
/home/s224075134/diffusion-forcing-tra

RuntimeError: No available kernel. Aborting execution.

In [3]:
algo.diffusion_model.model

Unet3D(
  (noise_level_pos_embedding): StochasticTimeEmbedding(
    (timesteps): StochasticUnknownTimesteps()
    (embedding): TimestepEmbedding(
      (linear_1): Linear(in_features=48, out_features=192, bias=True)
      (act): SiLU()
      (linear_2): Linear(in_features=192, out_features=192, bias=True)
    )
  )
  (init_conv): Conv3d(3, 48, kernel_size=(1, 7, 7), stride=(1, 1, 1), padding=(0, 3, 3))
  (rotary_time_pos_embedding): RotaryEmbedding()
  (init_temporal_attn): UnetTemporalAttentionBlock(
    (wrapper): EinopsWrapper(
      (module): TemporalAttentionBlock(
        (attn_block): AttentionBlock(
          (norm): LayerNorm((48,), eps=1e-05, elementwise_affine=True)
          (attn): Attention(
            (rotary_emb): RotaryEmbedding()
            (to_qkv): Linear(in_features=48, out_features=384, bias=False)
            (to_out): Linear(in_features=128, out_features=48, bias=True)
          )
        )
      )
    )
  )
  (down_blocks): ModuleList(
    (0): ModuleList(
  